<a href="https://colab.research.google.com/github/ThreeMachineExpression/contrapuntalPoetry/blob/main/PhoneticSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Parts of this notebook are based on Max Woolf's aitextgen notebook, as updated by Allison Parrish.

Also makes use of Kyle Gorman's syllabification library, syllabify.

Initial implementation looked for slant rhyme / slant alliteration by grabbing all of the consonants from the boundaries between vowels and summing their features, but that leads to too-mushy results. The syllabification approach misses assonance that crosses syllable boundaries - a todo is to figure out how to bring some of that back.

### Setup

In [ ]:
# Freeze versions of dependencies for now
!pip install tensorflow==1.15.0 keras==2.2.5 "h5py<3.0.0"
!pip3 install pytorch-lightning==0.7.6
!pip3 install transformers==2.9.1
!pip3 install fire==0.3.0

!pip install -q aitextgen==0.2.3

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

mount_gdrive()

!pip install annoy
!pip install pronouncing
!pip install pincelate
import pronouncing
import pincelate
pin = pincelate.Pincelate()

from collections import Counter

import random
import textwrap
import numpy as np

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

kPhoneticSimilarityVectorsRepo = "https://github.com/aparrish/phonetic-similarity-vectors"

!git clone {kPhoneticSimilarityVectorsRepo}

!python -m spacy download en_core_web_md

%cd phonetic-similarity-vectors
from featurephone import phone_feature_map as pfm
%cd ..

kSyllabifyRepo = "https://github.com/kylebgorman/syllabify"
!git clone {kSyllabifyRepo}

%cd syllabify
import syllabify
%cd ..

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 337kB 50.4MB/s 
     |████████████████████████████████| 512kB 44.7MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 3.8MB 36.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f0a04c0816b44cac94e437589db4298ca7da2a3687e9a335614b19f46b726a7e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.0
    Un

Using TensorFlow backend.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Cloning into 'phonetic-similarity-vectors'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.
     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051304 sha256=c0365e7a00dbb8effa8d9ff65d4c604ab9341b46ab6472893de2b5a571e4d15c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_vbtw9zu/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
/content/phonetic-similarity-vectors
/content
Cloning into 'syllabify'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
/content/syllabify
/content


In [ ]:
from_folder = "aitextgenDreamFineTuning"

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

In [ ]:
ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)

INFO:aitextgen:Loading GPT-2 model from provided pytorch_model.bin.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


# Generation

In [ ]:
!nvidia-smi

Sat Jan 23 22:21:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    41W / 300W |   1931MiB / 16130MiB |     10%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ai.generate()


You've wanted to create a story for a while only to have it leak out and into the void, where nothing real happens.
And then it's on the news instead of the people you know.
You try to turn it off for a while, but everyone keeps saying stupid things and it doesn't really work.
And then it all just goes back to pretending that everything's okay and everything's okay.
Like when the bright ball of light descends and lands in your throat and you can sing and everyone knows they're loved, the elderly women who are finally marrying feel in their bones that everything's okay and they really do love me.
I don't need anyone to love me anymore, I need someone to love me, and I NEEDED to hear that everyone was loved so much before.
I went to medical school so that I can provide proper care to the people I've hit over the head with 18th-century furniture.
Everything I need to know about this job interview is in the open field outside the hospital.
When I apply for this gig, there are rules agains

# Phonetic Similarity

In [ ]:
# Strategy:
# Split text into syllable buckets
# In each bucket, include the vowel sound, stress, and all of the bordering phonemes
#  (so a consonant between syllables goes in both buckets)
# Measure the distance between two buckets as 
#   a * (manhattan distance of consonant features)
# + b * (manhattan distance of vowel features)
# + c * (stress distance)
# (with a, b, c tuned as desired to emphasize different similarities)
#
# Pauses get a bucket of their own.

pfm['STOP'] = ()
stopSyllable = Syllable(([],['STOP0'],[]))

vowels = ('AO','AA','IY','UW','EH','IH','UH','AH','AE','EY','AY','OW','AW','OY',
          'ER','STOP')
consonantFeatures = ('alv','apr','asp','blb','dnt','frc','glt','lat','lbd','lbv','nas','pal','pla','stp','vcd','vel','vls')
vowelFeatures = ('bck','cnt','fnt','hgh','lmd','low','mid','rnd','rzd','smh','umd','unr','vwl')

# When deciding how close two syllables are, these constants indicate how much
# weight to give vowel features, stress, and consonant features
kDistanceWeightVowels = 4.0
kDistanceWeightStress = 5.0
kDistanceWeightOnset = 2.0
kDistanceWeightCoda = 3.0

class Syllable:
  # initialization takes a list of 3 lists of strings:
  # the phones for onset, nucleus, and coda
  # (as per the output of syllabify).
  #
  # vowel - vowel phone (w/o stress indicator) or STOP
  # stress - 0 for STOP, 1 for unstressed, 2 2ndary stress, 3 primary stress
  # (note this is different from the arpabet numbers)
  # ofeat - Counter containing a count of the consonant features in the onset
  # cfeat - Counter containing a count of the consonant features in the coda
  # vfeat - Counter containing a count of the vowel features
  def __init__(self, syllableList):
    self.vowel = syllableList[1][0][0:-1]

    # convert arpabet stress to a representation capable of similarity math
    stressConversion = {
        0: 1,
        1: 3,
        2: 2
    }

    if self.vowel = 'STOP':
      self.stress = 0
    else:
      self.stress = stressConversion[syllableList[1][0][-1]]
      
      self.ofeat = Counter()
      self.cfeat = Counter()
      self.vfeat = Counter()

      for feature in pfm[self.vowel]:
        self.vfeat[feature] += 1
      
      for phone in syllableList[0]:
        for feature in pfm[phone]:
          self.ofeat[feature] += 1
      
      for phone in syllableList[2]:
        for feature in pfm[phone]:
          self.cfeat[feature] += 1

def phones_for_word_fb(word):
  """Phones for word - either 1st entry in the CMU pronouncing dictionary
  or fallback to Pincelate sound-out if it's not in the dictionary.

  Takes the first pronounciation in the dictionary if there are multiple.

  Returns a list of phones.

  word - lowercase word, no spaces or punctuation
  """
  try:
    return pronouncing.phones_for_word(word)[0].split()
  except IndexError:
    return pin.soundout(word)

def syllablesFromText(string):
  """Turns text into a list of Syllable objects.

  One stop for ,; two stops for .:?! or double dash.
  Turns a hyphen between letters (as in "use-case") into a space,
   but in "a phrase - like this one" replaces it with a STOP

  Discards all other punctuation and non-alpha characters (including numbers).

  string -- text to convert to phones"""

  lidx = ridx = 0
  
  syllables = []
  doublestops = ['.','?','!',':','--']
  singlestops = [',',';','- ']

  while (lidx < len(string)):
    if string[lidx].isalpha():
      while ((ridx < len(string)) and (string[ridx].isalpha())):
        ridx += 1
      for s in syllabify.syllabify(phones_for_word_fb(string[lidx:ridx].lower())):
        syllables.append(Syllable(s))
      lidx = ridx
    else:
      while ((ridx < len(string)) and (not string[ridx].isalpha())):
        ridx += 1
      doublestop = False
      if any(c in string[lidx:ridx] for c in doublestops):
        syllables.extend([stopSyllable, stopSyllable])
      else:
        if any(c in stringp[lidx:ridx] for c in singlestops):
          syllables.append(stopSyllable)
      lidx = ridx

  return syllables


                
def distance(a : Syllable, b : Syllable):
  """A distance metric between two Syllable objects.
  Adjusted by the constant weights in the first cell,
  applied to Manhattan distance on features.
  """ 
  dist = abs(a.stress - b.stress) * kDistanceWeightStress

  for f in consonantFeatures:
    dist += abs(a.ofeat[f] - b.ofeat[f]) * kDistanceWeightOnset
    dist += abs(a.cfeat[f] - b.cfeat[f]) * kDistanceWeightCoda
  
  for f in vowelFeatures:
    dist += abs(a.vfeat[f] - b.vfeat[f]) * kDistanceWeightVowels

  return dist

def isVowel(phone):
  return phone[0:-1] in vowels

def canonFit(oldSyllables, newText, offset):
  """Given a syllable list and a batch of new text (as a string),
  returns the average syllable distance of newly added syllables
   to syllables *offset* syllables behind.
  Lower is rhymier.
  
  If newText doesn't turn up any new syllables, return a stupidly big number.

  TODO: new treatment of stops: total match for stop-to-stop, and don't count
  non-stop-to-stop comparisons toward the average.
  """

  newSyllables = syllabifier(arpabetPlusFromText(newText))

  if len(newSyllables == 0):
    return float("inf")

  # pull out the last syllable of the existing text, attach the onset of
  # 
  if len(oldSyllables > 0):
    newSyllables[0].onsetfeat += oldSyllables[-1].codafeat

  totalDist = 0
  i = -offset

  while (i + offset < len(oldSyllables) and i < len(list2)):
    totalDist += distance(list2[i], list1[i + offset])
    i = i + 1
  
  if i > 0:
    return totalDist/i
  else:
    # haven't hit the canon start point; don't influence generation at all
    return 0

def isascii(string):
  # Apparently this is a fast way of checking because the conversion
  # is implemented in C
  try:
    string.decode('ascii')
  except UnicodeDecodeError:
    return False
  else:
    return True

  return all(ord(c) < 128 for c in string)

In [ ]:
def generateCanon(alts = 100, tokensPerIncrement = 5, length = 100,
                  prompt = "If only I could be a little more",
                  temperature = 1.2, top_p = 0.9, offset = 10,
                  startingTokens=10):
  """
  alts - number of samples of text to generate to test for rhymes
  tokensPerIncrement - size of each incremental sample
  length - total tokens (including the prompt)
  prompt - start of canon
  temperature, top_p - passed along to GPT-2
  offset - how many syllables the 2nd part of the canon is delayed
  startingTokens - tokens for first generation. Should be more tokens than the
    prompt (tricky, because it's not clear how many tokens the prompt will contain)
  """
  print(prompt)
  canonSoFar = prompt
  seed = 0

  

  for i in range(startingTokens, length, tokensPerIncrement):
    canonSoFarSyllables = syllabifier(arpabetPlusFromText(canonSoFar))  
    bestTrial = canonSoFar
    bestDist = float("inf")

    for j in range(alts):
      seed += 1
      random.seed(seed)
      trial = ai.generate_one(
              prompt=canonSoFar,
              max_length=i,
              temperature=temperature,
              top_p=top_p)
      trialDiff = trial[len(canonSoFar):]
      if isascii(trialDiff): # only test generated text that our pronounciation lookups can handle
        print("next text: " + trialDiff)
        trialSyllables = canonSoFarSyllables + syllabifier(arpabetPlusFromText(trialDiff))
        if len(trialSyllables) > len(canonSoFarSyllables): # only test generations with real new text
          trialFit = canonFit(trialSyllables, canonSoFarSyllables, offset)
          if trialFit < bestDist:
            bestDist = trialFit
            bestTrial = trial
    
    canonSoFar = bestTrial
    print(canonSoFar)

  print()
  print("FINAL CANON:")
  print(canonSoFar)










In [ ]:
generateCanon(alts = 50, tokensPerIncrement=2, length=200, prompt="If only I could be", offset = 4, temperature = 1, top_p = 0.9)


If only I could be
next text:  your mother.
Oh
next text:  beautiful.
I don
next text:  her.
I'm
next text:  myself, and not for
next text:  the one to buy it
next text:  in a relationship with him
next text:  saved.
I'm
next text:  your father!
You
next text:  beautiful.
Love was
next text:  my best self.

next text:  beautiful.
That part
next text:  as good as this.
next text:  here.
I'd
next text:  like her.
[
next text:  beautiful and not have my
next text:  a little more careful.
next text:  like her!
You
next text:  saved.
I can
next text:  in a meeting coordinating this
next text:  saved!
I have
next text:  more beautiful.
And
next text:  the one to teach these
next text:  beautiful.
And if
next text:  myself.
It's
next text:  beautiful.
And you
next text:  your father.
Oh
next text:  a little better!

next text:  saved!
I thought
next text:  a little more careful.
next text:  your friend so quick."
next text:  like these people.

next text:  more beautiful.
I
next text:  my mot

KeyboardInterrupt: ignored



```
If  only I  could be in that  boat.       That boat.
            If    only  I     could be in that boat.

I    don't know why.       Cousinwog, cousin of  Brassy and
That boat.         I don't know why.  Cousinwog,     cousin 

Red from    Cousinwog  and   Mighty from   Cousinwog  
of  Brassy     and Red from  Cousinwog    and Mighty from

City  and  Plateaus  aren't  together        anymore.   They're
Cousinwog  City      and     Plateaus aren't together anymore. 

having a breakout and Plateaus says "What's going    on    here?"
      They're having   a breakout    and    Plateaus says "What's

going on here?"
```

(offset of 4, temperature .7)




```
If only I  could be my good   editor     in one piece 
        If only     I  could  be my good editor

-- which, at least,   it   was, when  absolutely forced and wri-
editor   --  which at least,    it   was, when absolutely

ting   ridiculously    long chunks of crap chunks.
forced and writing ridiculously       long chunks of crap

Without the   ethics.          "It sucks."    Sometimes,
chunks.       Without  the  ethics.       "It sucks."

sometimes, it's just "it's difficult       to tell a
Sometimes, sometimes,      it's just "it's difficult

story"   as art,   or an illusion.    There's another,     ever
to tell  a  story" as art, or an illusion.       There's another,

another trance.  Sometimes it's "I don't  think it's
   ever another  trance.        Sometimes it's  "I

important"         as   entertainment.
don't think it's   important" as  entertainment.
```
(offset of 3, temperature 2.0, vowels 4, stress 3, consonants 5)


```
If only I could be allowed to  be loved.     Been  avoiding
          If    only  I  could be allowed to be    loved.

eye  doctor since  ninety-two.
Been avoiding      eye doctor  since ninety-two.
```

(offset of 4, temperature 2, vowels, stress, cons = 2,1,2; 3 tokens per increment)

```
If only I could be that good.
People say it - "It's true it isn't" - "It is", "It is true it is true" - "Dear @lukejchengnott, in what way?"
```

(same settings, but offset of 3)

```
If only
I could be
as good as
this.
I know I'm
doing ev
ery thing right.
          I
know I'm do
ing every
thing I can.
          I
know I'm do
ing every
thing I can.
          I
know I'm cap
able.
     I be
lieve I am
loved.
I believe
I am ent
ertaining
the impos
sible.
  But all
of a   sud
den it seems
like it's a
bout to col
lapse on it
self and I
have to find
a place to
stand,  so
I don't lis
ten.
Finally
I find a
place to sit,
   thinking
it's okay
to be there.
      There
is a rea
sonable
place to go,
    but I'm
worried it
will be rai
ded by the
big demon
ic fuckups,
   and killed.
          I
know that I
am allowed
to feel my
anger and
anxie
ty under
control,
that this is
an extreme
ly danger
ous place.
    I sit
on it,
thinking it's
really o
kay to be
there.
But then I
get really
worried and
angry   and
a     pile  of
Trash piles up.
```

(alts -> 50 from 250, 5 tokens per increment, temperature 1)

(stress weight 1->200, offset -> 3)

In [ ]:
kDistanceWeightVowels = 2.0
kDistanceWeightStress = 5.0
kDistanceWeightConsonants = 5.0

s = syllabifier(arpabetPlusFromText("the road the road the road the road"))
canonFit(s,s,2)

30.0

In [ ]:
kDistanceWeightVowels = 4.0
kDistanceWeightStress = 5.0
kDistanceWeightOnset = 2.0
kDistanceWeightCoda = 3.0


In [22]:
syllabify.syllabify(arpabetPlusFromText("ugly"))

[([], ['AH1'], []), (['G', 'L'], ['IY0'], [])]